In [1]:
from gsamllavanav.cityreferobject import get_city_refer_objects, filter_objects_with_description

objects = filter_objects_with_description(get_city_refer_objects())

In [2]:
from collections import defaultdict
from tqdm import tqdm

object_names = defaultdict(set)

for map_name, single_map_objects in tqdm(objects.items(), desc='maps', position=0):
    for obj_id, obj in tqdm(single_map_objects.items(), desc='objects', position=1, leave=False):
        for desc_id, desc in enumerate(obj.processed_descriptions):
            object_names[map_name].add(desc.target)
            object_names[map_name].update(desc.surroundings)

maps: 100%|██████████| 34/34 [00:00<00:00, 320.21it/s]


In [5]:

import warnings
import numpy as np
from gsamllavanav.observation import cropclient
from gsamllavanav.maps.gsam_map import GSamMap, GSamParams
from gsamllavanav.subblocks import _split_map
from gsamllavanav.defaultpaths import GSAM_MAPS_DIR

gsam_maps_by_map = defaultdict(dict)
cropclient.load_image_cache()
warnings.filterwarnings('ignore')

ALTITUDE = 50
MAP_SIZE = 240
MAP_METERS = 410

dirname = GSAM_MAPS_DIR/f"full_scan/{(ALTITUDE, MAP_SIZE, MAP_METERS)}"
dirname.mkdir(exist_ok=True)

def gsam_scan_map(map_name: str, caption: str):
    gsam_map = GSamMap(map_name, (MAP_SIZE, MAP_SIZE), MAP_SIZE/MAP_METERS, [caption], GSamParams(True, 0.2))

    if not caption:
        return gsam_map.to_array(np.float16)

    for pose in _split_map(map_name, ALTITUDE):
        rgb = cropclient.crop_image(map_name, pose, (ALTITUDE*10, ALTITUDE*10), 'rgb')
        if (rgb > 0).mean() > 0.15:
            gsam_map.update_observation(pose, rgb)
    return gsam_map.to_array(np.float16)

for map_name, single_map_names in tqdm(object_names.items(), desc='maps', position=0):
    for caption in tqdm(single_map_names, desc='names', position=1, leave=False):
        caption : str = caption.replace('/', ' ')
        if (filename := dirname/f"{map_name}-{caption}.npy").exists():
            continue
        np.save(filename, gsam_scan_map(map_name, caption))

maps: 100%|██████████| 34/34 [22:17:06<00:00, 2359.61s/it]   
